# Langchain agents: multiple tools

In [1]:
from dotenv import load_dotenv
import os
import json
import re
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Tools
Let's provide two tools: a RAG retriever for Opentrons documentation and tool for initializing the deck layout for a JSON config file.

In [2]:
# Initialize vector database of Opentrons documentation

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("../rag-documentation/opentrons-docs.pdf")
documents = loader.load()

def clean_text(text):
    # Remove single newlines
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)
    # Replace multiple newlines with a single newline
    text = re.sub(r'\n+', '\n', text)
    return text.strip()

cleaned_docs = [Document(page_content=clean_text(doc.page_content), metadata=doc.metadata) for doc in documents]
splits = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200
        ).split_documents(cleaned_docs)
vector = FAISS.from_documents(splits, OpenAIEmbeddings())
retriever = vector.as_retriever(search_kwargs={"k": 3})

In [3]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "search_opentrons_docs",
    "Search for documentation about the Python Opentrons v2 API",
)

In [4]:
from langchain.tools import StructuredTool
from langchain_core.pydantic_v1 import BaseModel, Field

class DeckConfig(BaseModel):
    config_path: str = Field(..., description="A path to a JSON file containing an Opentrons deck layout")

def initialize_deck(config_path: str) -> str:
    """
    Initialize the Opentrons deck layout based on the provided JSON configuration.
    
    Args:
        config_string: JSON string representation of the deck layout.
    
    Returns:
        str: Python code to set up the Opentrons deck.
    """
    with open(config_path, 'r') as file:
        opentrons_config_string = file.read()

    config = json.loads(opentrons_config_string)
    code = []
    
    code.append("from opentrons import protocol_api")
    code.append(f"\nmetadata = {{'apiLevel': '{config['api_version']}'}}")
    code.append("\ndef run(protocol: protocol_api.ProtocolContext):")
    
    # Load pipettes
    for mount, pipette in config['pipettes'].items():
        tip_racks = [f"protocol.load_labware('{config['labware'][rack]['type']}', '{rack}')" 
                     for rack in pipette['tip_racks']]
        tip_racks_str = ", ".join(tip_racks)
        code.append(f"    {mount}_pipette = protocol.load_instrument('{pipette['type']}', '{mount}', tip_racks=[{tip_racks_str}])")
    
    # Load modules
    for module_name, module_info in config.get('modules', {}).items():
        code.append(f"    {module_name} = protocol.load_module('{module_info['type']}', '{module_info['location'][0]}')")
    
    # Load labware
    for slot, labware in config['labware'].items():
        if 'parent_module' in labware:
            code.append(f"    {slot}_labware = {labware['parent_module']}.load_labware('{labware['type']}')")
        else:
            code.append(f"    {slot}_labware = protocol.load_labware('{labware['type']}', '{slot}')")
    
    return "\n".join(code)

initialize_deck_tool = StructuredTool.from_function(
    func=initialize_deck,
    name="InitializeDeck",
    description="Initialize the Opentrons deck layout based on a JSON file.",
    args_schema=DeckConfig
)

In [5]:
tools = [retriever_tool, initialize_deck_tool]

In [7]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)
from langchain.agents import AgentExecutor

model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

system_template = """
You are an AI assistant that writes Opentrons code to complete a user specified objective. Your task is to:
    1. Write code to initialize the Opentrons deck layout based on a JSON configuration file. Use the 'InitializeDeck' tool to accomplish this.
    2. Using the 'SearchOpentronsDocs' tool to search the Opentrons documentation for information related to the user-specified task.
    3. Return the generated Python code including the output for (1) plus code to achieve the user-specified task.

    Only use the 'Initialize Deck' tool, proving the path to the JSON file as input. Do not attempt to modify or add to the generated code.
"""

human_template = "{input}"

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    HumanMessagePromptTemplate.from_template(input_variables=["input"], template=human_template),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_openai_tools_agent(model, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
config_path = "opentrons_config.json"
agent_executor.invoke({"input": f"Set the temperature of a thermocycler block to 20 degC.\n\nPath to opentrons deck config:\n{config_path}"})



> Entering new AgentExecutor chain...

Invoking: `InitializeDeck` with `{'config_path': 'opentrons_config.json'}`


from opentrons import protocol_api

metadata = {'apiLevel': '2.17'}

def run(protocol: protocol_api.ProtocolContext):
    left_pipette = protocol.load_instrument('p300_single_gen2', 'left', tip_racks=[protocol.load_labware('opentrons_96_tiprack_300ul', '1')])
    right_pipette = protocol.load_instrument('p20_multi_gen2', 'right', tip_racks=[protocol.load_labware('opentrons_96_tiprack_20ul', '2')])
    thermocycler = protocol.load_module('thermocyclerModuleV2', '7')
    1_labware = protocol.load_labware('opentrons_96_tiprack_300ul', '1')
    2_labware = protocol.load_labware('opentrons_96_tiprack_20ul', '2')
    3_labware = protocol.load_labware('corning_96_wellplate_360ul_flat', '3')
    5_labware = protocol.load_labware('nest_12_reservoir_15ml', '5')
    7_labware = thermocycler.load_labware('nest_96_wellplate_100ul_pcr_full_skirt')
Invoking: `search_opentrons_docs` wi

{'input': 'Set the temperature of a thermocycler block to 20 degC.\n\nPath to opentrons deck config:\nopentrons_config.json',
 'output': "Here is the generated Python code to initialize the Opentrons deck layout and set the temperature of the thermocycler block to 20 °C:\n\n```python\nfrom opentrons import protocol_api\n\nmetadata = {'apiLevel': '2.17'}\n\ndef run(protocol: protocol_api.ProtocolContext):\n    # Initialize the deck layout\n    left_pipette = protocol.load_instrument('p300_single_gen2', 'left', tip_racks=[protocol.load_labware('opentrons_96_tiprack_300ul', '1')])\n    right_pipette = protocol.load_instrument('p20_multi_gen2', 'right', tip_racks=[protocol.load_labware('opentrons_96_tiprack_20ul', '2')])\n    thermocycler = protocol.load_module('thermocyclerModuleV2', '7')\n    labware_1 = protocol.load_labware('opentrons_96_tiprack_300ul', '1')\n    labware_2 = protocol.load_labware('opentrons_96_tiprack_20ul', '2')\n    labware_3 = protocol.load_labware('corning_96_wellp